In [2]:
# imports
from fastai.text import *
import html

In [3]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('/home/jovyan/data/aclImdb/')

In [4]:
# standardize format
CLAS_PATH=Path('/home/jovyan/data/imdb_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('/home/jovyan/data/imdb_lm/')
LM_PATH.mkdir(exist_ok=True)

In [5]:
CLASSES = ['neg', 'pos', 'unsup']

In [21]:
!ls /home/jovyan/data/aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [27]:
for idx, label in enumerate(CLASSES):
    print(label, len(list((PATH/'train'/label).glob('*.*'))))
        

neg 12500
pos 12500
unsup 50000


In [39]:
def get_texts(path, frac: int=1):
    texts, labels = [], []
    for idx, label in enumerate(CLASSES):
        for n, fname in enumerate((path/label).glob('*.*')):
            texts.append(fname.open('r').read())
            labels.append(idx)
            if n == len(list((path/label).glob('*.*')))//frac:
                break
    return np.array(texts), np.array(labels)

In [40]:
keep_frac = 1000

In [41]:
trn_texts, trn_labels = get_texts(PATH/'train', keep_frac)

In [42]:
val_texts, val_labels = get_texts(PATH/'test', keep_frac)

In [43]:
len(trn_texts), len(val_texts)

(77, 26)

In [44]:
col_names = ['labels', 'text']

In [45]:
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))
val_idx = np.random.permutation(len(val_texts))

In [46]:
trn_texts = trn_texts[trn_idx]
val_texts = val_texts[val_idx]

In [47]:
trn_labels = trn_labels[trn_idx]
val_labels = val_labels[val_idx]

In [57]:
df_trn = pd.DataFrame({'text': trn_texts, 'labels': trn_labels}, columns=col_names)
df_val = pd.DataFrame({'text': val_texts, 'labels': val_labels}, columns=col_names)

In [61]:
# Yann LeCun-style NLP data saving
df_trn[df_trn['labels'] != 2].to_csv(CLAS_PATH/'train.csv', header=False, index=False)
df_val.to_csv(CLAS_PATH/'val.csv', header=False, index=False)

(CLAS_PATH/'classes.txt').open('w').writelines(f'{o}\n' for o in CLASSES)

In [62]:
trn_texts, val_texts = sklearn.model_selection.train_test_split(np.concatenate([trn_texts, val_texts]), test_size=0.1)

In [63]:
len(trn_texts), len(val_texts)

(92, 11)

In [64]:
# save again
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)